In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from typing import List
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Вспомогательные функции

In [ ]:
def metric(y_true, y_pred) -> float:
    return float(np.sum(np.abs(y_true - y_pred)) / np.sum(y_pred) * 100)

In [ ]:
def indicies_of_outliers(x):
    ql, q3 = np.percentile(x, [25, 75])
    iqr = q3 - ql
    lower_bound = ql - (iqr * 1.5)
    upper_bound = q3 + (iqr * 1.5)
    return np.where((x > upper_bound) | (x < lower_bound))

In [ ]:
def identify_collinear(X: pd.DataFrame,
                       correlation_threshold: float) -> List[str]:
    '''
        Функция поиск корреляции признаков
        На вход принимает фрейм независимых переменных и значение для корреляции
        Возвращает список признаком с высокой корреляцией
    '''
    corr_matrix = X.corr()

    upper = corr_matrix.where(
        np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

    to_drop = [
        column for column in upper.columns
        if any(upper[column].abs() > correlation_threshold)
    ]

    record_collinear = pd.DataFrame(
        columns=['drop_feature', 'corr_feature', 'corr_value'])

    for column in to_drop:
        corr_features = list(
            upper.index[upper[column].abs() > correlation_threshold])
        corr_values = list(
            upper[column][upper[column].abs() > correlation_threshold])

        drop_features = [column for _ in range(len(corr_features))]

        temp_df = pd.DataFrame.from_dict({
            'drop_feature': drop_features,
            'corr_feature': corr_features,
            'corr_value': corr_values
        })

        record_collinear = record_collinear.append(temp_df, ignore_index=True)

    record_collinear = record_collinear
    return to_drop

# Исследование данных

In [ ]:
df = pd.read_csv('data_regression_for_task.csv', sep=',')
df.head()

In [ ]:
# Посмотрим на уникальные значения в каждом признаке
for i in df.columns:
    x = df[i].value_counts()
    print('column', i, 'values', len(x))

In [ ]:
# YEAR - признак не информативен, тк принимает только 2 разных значение -> удалим его
# MONTH
# CONTRAGENT - предположим, признак имеет какую-то естественную упорядочность (индекс контрагента который поставляет товар соответсвует порядковому номеру начала работы с магазином)
# ARTICLE_CODE - похож на уникальный номер товара
# ARTICLE_NAME - расшифровка поля ARTICLE_CODE -> удалим его
# ARTICLE_GROUP - группа товаров -> сделаем кодировку номинальных признаков
# SALES -> проведем стандартизацию признаков
# STORE_SALES -> проведем стандартизацию признаков

In [ ]:
# Удалим пропущенные значения
df.dropna(inplace=True)

# Удалим столбцы: YEAR, ARTICLE_NAME
df.drop(['ARTICLE_NAME', 'YEAR'], inplace=True, axis=1)

In [ ]:
# Удалим выбросы не входящие в МКР
to_drop_index = indicies_of_outliers(df.STORE_SALES)[0].tolist()
df.drop(df.index[to_drop_index], inplace=True)

In [ ]:
# Закодируем номинальные категориальный признак
df = pd.get_dummies(df, columns=['ARTICLE_GROUP', 'MONTH'])

In [ ]:
test_last_month = df[df.MONTH_12 == 1]

X_test_last_month = test_last_month[['CONTRAGENT', 'ARTICLE_CODE', 'STORE_SALES',
       'ARTICLE_GROUP_BEER', 'ARTICLE_GROUP_KEGS', 'ARTICLE_GROUP_LIQUOR',
       'ARTICLE_GROUP_NON-ALCOHOL', 'ARTICLE_GROUP_REF',
       'ARTICLE_GROUP_STR_SUPPLIES', 'ARTICLE_GROUP_WINE', 'MONTH_1',
       'MONTH_2', 'MONTH_4', 'MONTH_5', 'MONTH_6', 'MONTH_8', 'MONTH_9',
       'MONTH_10', 'MONTH_11', 'MONTH_12']]
y_test_last_month = test_last_month[['SALES']]

# Закодиурем порядковые категориальные признаки
cc = LabelEncoder().fit(X_test_last_month.CONTRAGENT)
X_test_last_month['CONTRAGENT_CODE'] = cc.transform(X_test_last_month.CONTRAGENT)
X_test_last_month.drop('CONTRAGENT', axis=1, inplace=True)

# Уберем символы из признака и преобразуем в целому числу(положительному)
X_test_last_month['ARTICLE_CODE'] = X_test_last_month['ARTICLE_CODE'].str.replace(r'\D', '')
X_test_last_month.ARTICLE_CODE = X_test_last_month.ARTICLE_CODE.astype(np.uint64)

drop_col = identify_collinear(X_test_last_month, 0.7)
X_test_last_month.drop(drop_col, axis=1, inplace=True)

# Стандартизируем количественные данные
scaler = StandardScaler().fit_transform(X_test_last_month[['STORE_SALES']])
scaled_features_df = pd.DataFrame(scaler, index=X_test_last_month.index, columns=X_test_last_month[['STORE_SALES']].columns)
X_test_last_month.drop('STORE_SALES', axis=1, inplace=True)
X_test_last_month = X_test_last_month.join(scaled_features_df)

In [ ]:
X = df[['CONTRAGENT', 'ARTICLE_CODE', 'STORE_SALES',
       'ARTICLE_GROUP_BEER', 'ARTICLE_GROUP_KEGS', 'ARTICLE_GROUP_LIQUOR',
       'ARTICLE_GROUP_NON-ALCOHOL', 'ARTICLE_GROUP_REF',
       'ARTICLE_GROUP_STR_SUPPLIES', 'ARTICLE_GROUP_WINE', 'MONTH_1',
       'MONTH_2', 'MONTH_4', 'MONTH_5', 'MONTH_6', 'MONTH_8', 'MONTH_9',
       'MONTH_10', 'MONTH_11', 'MONTH_12']]
y = df[['SALES']]

In [ ]:
# Закодиурем порядковые категориальные признаки
cc = LabelEncoder().fit(X.CONTRAGENT)
X['CONTRAGENT_CODE'] = cc.transform(X.CONTRAGENT)
X.drop('CONTRAGENT', axis=1, inplace=True)

In [ ]:
# Уберем символы из признака и преобразуем в целому числу(положительному)
X['ARTICLE_CODE'] = X['ARTICLE_CODE'].str.replace(r'\D', '')
X.ARTICLE_CODE = X.ARTICLE_CODE.astype(np.uint64)

In [ ]:
# Построим корреляционную матрицу и удалим признаки с корреляцией > 0.7 по модулю
plt.figure(figsize=(15, 8))
ax = sns.heatmap(X.corr(), cmap='viridis', annot=True, linewidths=.5)
drop_col = identify_collinear(X, 0.7)
print(f'Признаки которые были удалены\n{drop_col}')
X.drop(drop_col, axis=1, inplace=True)

In [ ]:
# Стандартизируем количественные данные
scaler = StandardScaler().fit_transform(X[['STORE_SALES']])
scaled_features_df = pd.DataFrame(scaler, index=X.index, columns=X[['STORE_SALES']].columns)
X.drop('STORE_SALES', axis=1, inplace=True)
X = X.join(scaled_features_df)

# Посмтроение модели

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
## Сначала посмотрим как ведут себя основные модели без настроки гиперпараметров

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

models = [
    LinearRegression(),
    DecisionTreeRegressor(),
    KNeighborsRegressor(),
    RandomForestRegressor()
]

TestModel = pd.DataFrame()
temp_dict = {}

for model in models:
    m = str(model)
    model.fit(X_train, y_train)
    temp_dict['Model'] = m[:m.index('(')]
    temp_dict['R^2_test'] = metric(y_test,
                                   model.predict(X_test).reshape(-1, 1))
    TestModel = TestModel.append([temp_dict])

TestModel = TestModel.set_index(['Model'])
TestModel

## Произведем поиск по сетки гиперпараметров для к ближайщих соседей и случаного леса

In [ ]:
def grid_search_for_model(model, params):
    grid_search = GridSearchCV(estimator=model,
                              param_grid=params,
                              cv=5,
                              verbose=1)
    grid_search.fit(X_train, y_train.values.ravel())
    return grid_search.best_params_, grid_search.cv_results_['params']

In [ ]:
param_grid = {'n_neighbors': np.arange(1, 12, 2),
              'weights': ['uniform', 'distance']}

knn = KNeighborsRegressor()

best_params, params = grid_search_for_model(knn, param_grid)
best_params, params

In [ ]:
param_grid = {
    'n_estimators': list(range(50, 101, 10)),
    'max_depth': list(range(10, 15)),
    'max_features': list(range(5, 10)),
}

rf = RandomForestRegressor()

best_params, params = grid_search_for_model(rf, param_grid)
 best_params, params

## Обучим случайный лес с найденными гиперпараметрами и оценим точность на последнем месяце
### * По хороше надо сделать кросвалидацию, не нашел как в cros_val_score засунусь свою метрику сделал на r2

In [ ]:
rf = RandomForestRegressor(random_state=42,
                           max_depth=best_params['max_depth'],
                           max_features=best_params['max_features'],
                           n_estimators=best_params['n_estimators'])
rf.fit(X_train, y_train.values.ravel())
y_pred = rf.predict(X_test_last_month)
print(f'Оценка точности на последний месяц: {round(metric(y_test_last_month, y_pred.reshape(-1, 1)),2)}')

In [ ]:
scores = cross_val_score(rf, X, y.values.ravel(), cv=5, scoring='r2')
print(f'R^2 на перекрестной проверки: {scores}')
print(f'Mean R^2:\t {round(np.mean(scores), 2)}')